# Проект по курсу "Рекомендательные системы"
  
Правила заполнения ноутбуков на авто-проверку:
- повторить окружение преподавателя
Python 3.13.0
```bash
pip install implicit==0.7.2 "rectools[all]==0.17.0" pandas==2.3.3 numpy==2.3.3 scipy==1.16.2  requests==2.32.5 catboost==1.2.8 scikit-learn==1.7.2
```
- все решение должно полностью помещаться в функцию solution(смотри пример). Если вы хотите реализовать дополнительные функции - поместите их в область видимости soluition. Нельзя использовать дополнительные файлы.
- не добавлять новые импорты и не использовать дополнительные библиотеки. В противном случае ноутбук не пройдёт проверку и получит `0` баллов
- не добавлять аргументов в solution
- писать код только между # CODE BEGIN и # CODE END
- не менять код преподавателя
- не добавлять новые ячейки
- следить, чтобы не было warning - они автоматом фейлят задание
- перед сдачей проверить, что весь ноутбук прогоняется от начала до конца и все тесты проходят
- data_path должен браться из переменной окружения как в коде ниже
- Код должен выполняться за разумное время - ограничение 20 мин на 4 CPU и 16 Gb RAM без GPU. Не нужно ставить огромное количество эпох.
- Постарайтесь максимально зафиксировать сиды, чтобы не было сюрпризов во время автоматической проверки. В случае, если решение выдает разное качество при разных запусках, то в зачет идет то значение, которое получилось при автоматической проверке.


В данном проекте вам возможно захочется подбирать гипер-параметры моделей. Писать код для подбора гипер-параметров, использовать optuna и т.п. рекомендуем в отдельном ноутбуке.

Библиотеки implicit и lightfm не фиксируют random state при num_threads > 1. Если результат работы модели не сильно превышает  необходимый порог и рандом может опустить его ниже требуемого уровня, рекомендуем продолжить повышение качества модели: тюнинг гипер-параметров, подбор фичей, подбор метода обработки датасета

In [5]:
# Установка всех необходимых библиотек (точные версии из требований проекта)
# ВНИМАНИЕ: Перед финальной сдачей удалите эту ячейку (правило: не добавлять новые ячейки)
%pip install implicit==0.7.2 "rectools[all]==0.17.0" pandas==2.3.3 numpy==2.3.3 scipy==1.16.2 requests==2.32.5 catboost==1.2.8 scikit-learn==1.7.2 threadpoolctl


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached scipy-1.16.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata 

# Задание

Вам предлагается реализовать рекомендательную систему для фильмов KION.
Решение должно быть полностью упаковано в функцию solution. Качество будет проверяться с помощью метрики MAP@10 на отложенной неделе. Итоговый бал определеятся функцией scorer - вы можете посмотреть его сразу, но если модель не подразумевает фиксирование random state, то после прогона автоматической системой результат может немного отличаться.

В качестве примера реализована базовая рекомендательная система на основе ease. Ваша задача - улучшить эту систему.

Чтобы решение отрабатывало быстрее будем использовать 10% от общего числа пользователей.

В случае, если в вашем решении будет найден Hardcode элементов тестового датафрейма - работа будет аннулирована.

Напоминаю, что для зачета по курсу нужно набрать в сумме с доп баллами и первым дз 60 баллов.

Успехов!

Подсказки:
- Можно посмотреть документацию rectools
- Можно поссмотреть ноутбуки с семинаров и предыдущую версию проекта
- Не стесняйтесь добавлять фичи в ранжирование
- Скорее всего вам понадобятся как отбор кандидатов, так и ранжирование

## Импорты и данные

In [6]:
!python -V

Python 3.13.0


In [7]:
# Убедитесь, что вы не добавляете новые импорты в ноутбук. Решение должно быть ограничено данными библиотеками
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import warnings
warnings.simplefilter("ignore")

import implicit
import rectools
import pandas as pd
import numpy as np
import scipy
import requests
import catboost
import sklearn

from rectools import models
from rectools import dataset
from rectools import metrics

print(implicit.__version__)
print(rectools.__version__)
print(pd.__version__)
print(np.__version__)
print(scipy.__version__)
print(requests.__version__)
print(catboost.__version__)
print(sklearn.__version__)

0.7.2
0.17.0
2.3.3
2.3.3
1.16.2
2.32.5
1.2.8
1.7.2


In [8]:
import os.path

# For implicit ALS
import threadpoolctl
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

Если у вас нет данных, то используйте закомментированный код

In [9]:
# from tqdm.auto import tqdm
# import zipfile as zf

# url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

# req = requests.get(url, stream=True)

# with open('kion.zip', 'wb') as fd:
#     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
#     progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
#     for chunk in req.iter_content(chunk_size=2 ** 20):
#         progress_bar.update(len(chunk))
#         fd.write(chunk)

# files = zf.ZipFile('kion.zip', 'r')
# files.extractall()
# files.close()

In [10]:
data_path = os.environ.get("DATA_PATH")
if data_path is None:
    data_path = "data_original"  # ваш путь к данным до папки data_original включительно (поменяйте при необходимости)

In [11]:
users = pd.read_csv(os.path.join(data_path, "users.csv"))
items = pd.read_csv(os.path.join(data_path, "items.csv"))

users = users.sample(frac=0.1, random_state=42)

interactions = (
    pd.read_csv(os.path.join(data_path, "interactions.csv"), parse_dates=["last_watch_dt"])
    .rename(columns={'total_dur': rectools.Columns.Weight,
                     'last_watch_dt': rectools.Columns.Datetime})
)


interactions = interactions[interactions["user_id"].isin(users["user_id"])]


print(interactions.shape)
interactions.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'data_original/users.csv'

In [ ]:
N_DAYS = 7

max_date = interactions['datetime'].max()
train = interactions[(interactions['datetime'] <= max_date - pd.Timedelta(days=N_DAYS))]
test = interactions[(interactions['datetime'] > max_date - pd.Timedelta(days=N_DAYS))]

catalog = train[rectools.Columns.Item].unique()

test_users = test[rectools.Columns.User].unique()
cold_users = set(test_users) - set(train[rectools.Columns.User])
test.drop(test[test[rectools.Columns.User].isin(cold_users)].index, inplace=True)
hot_users = test[rectools.Columns.User].unique()
print(test.shape[0])
print(test[rectools.Columns.User].nunique())

def scorer(map: float):
    print(f"Ваш MAP: {map}")
    UPPER_BOUND = 0.089
    LOWER_BOUND = 0.071
    score = int(min(max( (map - LOWER_BOUND) / (UPPER_BOUND - LOWER_BOUND), 0), 1) * 80)
    print(f"Ваш итоговый балл: {score}")
    return score

24771
9099


In [ ]:
def solution(train: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame):
    #  CODE BEGIN
    def filter_dataset(train):
        train[rectools.Columns.Weight] = np.where(train['watched_pct'] > 10, 3, 1)
        dataset_ease = rectools.dataset.Dataset.construct(train)
        return dataset_ease
    
    dataset_ease = filter_dataset(train)
    ease = rectools.models.EASEModel()
    ease_model = ease.fit(dataset_ease)

    
    ease_res = ease_model.recommend(
        users=hot_users,
        dataset=dataset_ease,
        k=10,
        filter_viewed=True,
    )
    #  CODE END
    return ease_res


In [ ]:
%%time

recos = solution(train.copy(), users.copy(), items.copy())
scorer(rectools.metrics.MAP(10).calc(recos, test))

Ваш MAP: 0.03321778223372767
Ваш итоговый балл: 0
CPU times: user 37.2 s, sys: 3.38 s, total: 40.5 s
Wall time: 41.4 s


0